### Run this lines to read the csv file containing the data from the CSV file. 

In [21]:
import pandas as pd ## importing the pandas package 
Spirit= pd.read_csv('Spirit Airways.csv')
print(Spirit)

         Route  Q-class fare price  Y-class fare price  \
0  Bos-Chicago                 200                 230   
1       Bos-SF                 320                 420   
2       Bos-LA                 400                 490   
3   NY-Chicago                 250                 290   
4        NY-SF                 410                 550   
5        NY-LA                 450                 550   
6   Chicago-SF                 200                 230   
7   Chicago-LA                 250                 300   

   Expected demand for Q-class  Expected demand for Y-class  
0                           25                           20  
1                           55                           40  
2                           65                           25  
3                           24                           16  
4                           65                           50  
5                           40                           35  
6                           21             

In [2]:
# For example, to access column the prices for Q-class:
Spirit['Q-class fare price']

0    200
1    320
2    400
3    250
4    410
5    450
6    200
7    250
Name: Q-class fare price, dtype: int64

# Q1

### 1. For each query, determine the possible range of bids that Residence Inn can submit to land in each position.

For "hotel near McGill":
- **Le St Martin**: 125
- **Le Germain**: 90
- **Sofitel**: 120
- **Residence Inn**: 9 x b1

For "McGill hotel":
- **Sofitel**: 160
- **Le St Martin**: 75
- **Le Germain**: 0 
- **Residence Inn**: 9 x b2 

For "Downtown Montreal hotel":
- **Le St Martin**: 175
- **Le Germain**: 120
- **Sofitel**: 100
- **Residence Inn**: 9 x b3 

1. **first position** - Residence Inn's bid x QS must exceed all others.
2. **second position** - it must exceed all but the highest bid x QS.
3. **third position** - it must be higher than at least one other bid x QS.
4. **fourth position** - it simply needs to be positive

In [8]:
QS_Residence_Inn = 9

bids_and_quality_scores = {
    "Hotel": ["Residence Inn", "Le St Martin", "Le Germain", "Sofitel"],
    "hotel near McGill Bid xQS": [125, 90, 120],
    "McGill hotel Bid xQS": [75, 0, 160],
    "Downtown Montreal hotel Bid xQS": [175, 120, 100]
}

bids_near_mcgill = bids_and_quality_scores["hotel near McGill Bid xQS"]
bids_mcgill_hotel = bids_and_quality_scores["McGill hotel Bid xQS"]
bids_downtown_montreal = bids_and_quality_scores["Downtown Montreal hotel Bid xQS"]

def calculate_bid_ranges(bid_x_qs_list, QS):
    bid_x_qs_list_sorted = sorted(bid_x_qs_list, reverse=True)
    ranges = []
    for i in range(len(bid_x_qs_list_sorted)+1):
        if i == 0:  # First position
            lower_bound = (bid_x_qs_list_sorted[i]/QS) + 0.01  # Must bid higher than highest bid x QS
            upper_bound = float('inf')  # No upper limit for the first position
        elif i == len(bid_x_qs_list_sorted):  # Last position
            lower_bound = 0  # No lower limit for the last position
            upper_bound = (bid_x_qs_list_sorted[i-1]/QS) - 0.01  # Must bid less than the lowest bid x QS
        else:
            lower_bound = (bid_x_qs_list_sorted[i]/QS) + 0.01  # Must bid higher than the next lower bid x QS
            upper_bound = (bid_x_qs_list_sorted[i-1]/QS) - 0.01  # Must bid less than the next higher bid x QS
        ranges.append((lower_bound, upper_bound))
    return ranges

bid_ranges_near_mcgill = calculate_bid_ranges(bids_near_mcgill, QS_Residence_Inn)
bid_ranges_mcgill_hotel = calculate_bid_ranges(bids_mcgill_hotel, QS_Residence_Inn)
bid_ranges_downtown_montreal = calculate_bid_ranges(bids_downtown_montreal, QS_Residence_Inn)

bid_ranges_near_mcgill, bid_ranges_mcgill_hotel, bid_ranges_downtown_montreal


([(13.898888888888889, inf),
  (13.343333333333334, 13.87888888888889),
  (10.01, 13.323333333333334),
  (0, 9.99)],
 [(17.78777777777778, inf),
  (8.343333333333334, 17.767777777777777),
  (0.01, 8.323333333333334),
  (0, -0.01)],
 [(19.454444444444444, inf),
  (13.343333333333334, 19.43444444444444),
  (11.12111111111111, 13.323333333333334),
  (0, 11.101111111111111)])

For the query "hotel near McGill":
- First Position: Bid must be higher than $13.90.
- Second Position: Bid must be between $13.34 and $13.88.
- Third Position: Bid must be between $10.01 and $13.32.
- Fourth Position: Bid must be less than $9.99.

For the query "McGill hotel":
- First Position: Bid must be higher than $17.79.
- Second Position: Bid must be between $8.34 and $17.76.
- Third Position: Bid must be between $0.01 and $8.32.
- Fourth Position: This is not applicable as Le Germain has bid 0, making it impossible to bid lower while maintaining a positive bid.

For the query "Downtown Montreal hotel":
- First Position: Bid must be higher than $19.45.
- Second Position: Bid must be between $13.34 and $19.43.
- Third Position: Bid must be between $11.12 and $13.32.
- Fourth Position: Bid must be less than $11.10.

### 2. For each query and position, what is the PPC (pay per click) for the Residence Inn?

- **"hotel near McGill"** 
    - First Position: $0.01$
    - Second Position: $\frac{125}{9} + 0.01 \approx 13.89$
    - Third Position: $\frac{90}{9} + 0.01 = 10.01$
    - Fourth Position: $\frac{120}{9} + 0.01 \approx 13.34$
    

- **"McGill hotel"** 
    - First Position: $0.01$
    - Second Position: $\frac{75}{9} + 0.01 \approx 8.34$
    - Third Position: $0$
    - Fourth Position: $\frac{160}{9} + 0.01 = 17.78$

- **"Downtown Montreal hotel"** 
    - First Position PPC: $0.01$
    - Second Position PPC: $\frac{175}{9} + 0.01 \approx 19.54$
    - Third Position: $\frac{120}{9} + 0.01 \approx 13.34$
    - Fourth Position: $\frac{100}{9} + 0.01 \approx 11.12$.

### 3. Residence Inn's bid optimization problem is reduced to deciding which position they want to be in for every query. Residence Inn will always bid for one position only per query. Definne your variables. Write the constraint that your variables need to verify.

### Decision Variables

Let $x_{i,j}$ be the decision variable that indicates whether Residence Inn chooses position $j$ for query $i$, where $i$ is the query index, and $j$ is the position index. For simplicity, $x_{i,j}$ can be binary:

- $x_{i,j} = 1$ if Residence Inn chooses position $j$ for query $i$,
- $x_{i,j} = 0$ otherwise.


### Constraints

1. **Single Position per Query Constraint**: For each query $i$, the Residence Inn can only choose one position. This can be represented as:
$$  \sum_{j} x_{i,j} = 1 \quad \forall i  $$

2. **Binary Constraint**: The decision variables are binary:
$$ x_{i,j} \in \{0,1\} \quad \forall i,j $$

4. **Non-Negativity Constraint**: Although already covered by the binary nature of $x_{i,j}$, it's implicitly stated that:
$$ x_{i,j} \geq 0 \quad \forall i,j $$


### 4.Residence Inn has set a maximum number of times (Max Displays, see Figure 3) they would bid for any position for every query. Assume if Residence Inn bids for a particular position for a query, they will bid that maximum number of displays.

(a) Write the budget constraint for Residence Inn. \
(b) Write the Objective function for Residence Inn. 

#### (a) Budget Constraint for Residence Inn

Let $B$ represent Residence Inn's daily budget. The budget constraint is then:

$$ \sum_{i} \sum_{j} (PPC_{i,j} \cdot CTR_{j} \cdot MaxDisplays_{i,j} \cdot x_{i,j}) \leq B $$

### (b) Objective Function for Residence Inn

$$ \text{Maximize} \quad Z = \sum_{i} \sum_{j} (CTR_{j} \cdot MaxDisplays_{i,j} \cdot x_{i,j}) $$

### 5. Using Jupyter Notebook and Gurobi, write the Residence Inn's optimization problem that you for- mulated in the previous questions. What is the optimal bid position decisions for each query? What should the actual bid ranges for b1, b2 and b3 be? What is the optimal expected click-through rate for the Residence Inn?

In [1]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("Residence_Inn_Optimization")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-18


In [7]:
x = model.addVars(3, 4, vtype=GRB.BINARY, name="x")

In [8]:
for i in range(3):  # Assuming 3 queries
    model.addConstr(gp.quicksum(x[i, j] for j in range(4)) == 1, f"SinglePosition_{i}")

In [9]:
PPC = [[0.01, 13.89, 10.01, 13.34], [0.01, 8.34, 0, 17.78], [0.01, 19.54, 13.34, 11.12]]  # Placeholder PPC values
CTR = [0.08, 0.05, 0.025, 0.01]  # Placeholder CTR values
MaxDisplays = [[8, 14, 15, 15], [7, 20, 20, 0], [4, 11, 25, 25]]  # Placeholder MaxDisplays values

B = 10  # Residence Inn's budget

model.addConstr(
    sum(PPC[i][j] * CTR[j] * MaxDisplays[i][j] * x[i,j] for i in range(3) for j in range(4)) <= B, 
    "BudgetConstraint"
)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
model.setObjective(
    sum(CTR[j] * MaxDisplays[i][j] * x[i,j] for i in range(3) for j in range(4)), 
    GRB.MAXIMIZE
)

In [11]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12800H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 5 rows, 24 columns and 32 nonzeros
Model fingerprint: 0xe2c96882
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+01]
  Objective range  [1e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]

MIP start from previous solve produced solution with objective 1.96 (0.00s)
Loaded MIP start from previous solve with objective 1.96

Presolve removed 5 rows and 24 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 1.96 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.960000000000

In [12]:
for i in range(3):
    for j in range(4):
        if x[i,j].X > 0.5:  # If the decision variable is selected
            print(f"Query {i+1}, Position {j+1} selected.")

Query 1, Position 1 selected.
Query 2, Position 2 selected.
Query 3, Position 1 selected.


#### For "hotel near McGill" (Query 1, Position 1):
- To secure the first position, the bid must be higher than $13.90.

#### For "McGill hotel" (Query 2, Position 2):
- The bid must be between $8.34 and $17.76.

#### For "Downtown Montreal hotel" (Query 3, Position 1):
- To secure the first position, the bid must be higher than $19.45.

### Optimal Expected Click-Through Rate (CTR):

$ \text{Optimal Expected CTR} = (0.08 \times 8) + (0.05 \times 20) + (0.08 \times 4) = 1.96 $


### 6. Suppose instead that Residence Inn is allowed to vary its bids to have an array of positions per query (They no longer only bid on one position only), but are still subjected to a maximum number of displays per position per query they cannot surpass. Reformulate the Residence Inn Optimization problem

- $x_{i,j} \geq 0$ and integer 

The objective function remains to maximize the total expected number of clicks, but now considers the variable number of displays for each position:

$$ \text{Maximize} \quad Z = \sum_{i} \sum_{j} (CTR_{j} \cdot x_{i,j}) $$


# Q2

### Decision Variables:
- Let $Q1, Q2, \ldots, Q8$ represent the number of seats reserved for discounted Q-Class on the 8 routes.
- Let $Y1, Y2, \ldots, Y8$ represent the number of seats reserved for unrestricted Y-Class on the 8 routes.

### Objective Function:
The objective is to maximize total revenue, which is the sum of revenues from both classes across all 8 routes:

$$ \text{Maximize Revenue} = \sum_{i=1}^{8} (P_{Qi} \cdot Q_i + P_{Yi} \cdot Y_i) $$

where $P_{Qi}$ and $P_{Yi}$ are the prices of Q-Class and Y-Class tickets for the $i$th route, respectively.

### Constraints:

#### Capacity Constraints:
Each flight leg has a capacity constraint that the sum of Q-Class and Y-Class seats cannot exceed the total seats available on the aircraft.

For example, for the first route on aircraft 1:

$$ Q1 + Y1 \leq 140 $$

This constraint would be replicated for each route and aircraft.

#### Demand Constraints:
The number of seats reserved for each fare class cannot exceed the expected daily demand for that class on that route.

For example, for Q-Class on the first route:

$$ Q1 \leq D_{Q1} $$

where $D_{Q1}$ is the expected daily demand for Q-Class seats on the first route.

#### Non-Negativity Constraints:
All decision variables must be greater than or equal to zero, representing the number of seats.

$$ Q_i, Y_i \geq 0 \quad \forall i \in \{1, 2, \ldots, 8\} $$

#### Decision Variables Must Be Integers:
Since you cannot reserve a fraction of a seat, the decision variables must be integers.

$$ Q_i, Y_i \in \mathbb{Z} \quad \forall i \in \{1, 2, \ldots, 8\} $$

In [13]:
model = gp.Model("Spirit_Airways_Revenue_Optimization")

In [27]:
fare_prices_q = Spirit['Q-class fare price'].tolist()
fare_prices_y = Spirit['Y-class fare price'].tolist()
expected_demand_q = Spirit['Expected demand for Q-class'].tolist()
expected_demand_y = Spirit['Expected demand for Y-class'].tolist()

aircraft_capacity = 140

q_vars = model.addVars(8, name='Q', lb=0, ub=aircraft_capacity, vtype=GRB.CONTINUOUS)

y_vars = model.addVars(8, name='Y', lb=0, ub=aircraft_capacity, vtype=GRB.CONTINUOUS)

In [16]:
# Objective: Maximize revenue from both classes across all routes
model.setObjective(
    gp.quicksum(fare_prices_q[i] * q_vars[i] + fare_prices_y[i] * y_vars[i] for i in range(8)),
    GRB.MAXIMIZE
)


In [17]:
# Capacity constraints for each route
for i in range(8):
    model.addConstr(q_vars[i] + y_vars[i] <= aircraft_capacity, f"Capacity_{i}")

# Demand constraints for Q-Class
for i in range(8):
    model.addConstr(q_vars[i] <= expected_demand_q[i], f"Demand_Q_{i}")

# Demand constraints for Y-Class
for i in range(8):
    model.addConstr(y_vars[i] <= expected_demand_y[i], f"Demand_Y_{i}")


In [18]:
# Solve model
model.optimize()


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12800H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24 rows, 32 columns and 32 nonzeros
Model fingerprint: 0x2faaf8bf
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 6e+02]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e+01, 1e+02]
Presolve removed 24 rows and 32 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0354000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.035400000e+05


In [19]:
if model.status == GRB.OPTIMAL:
    print("Optimal Allocation Decisions for Each Flight and Class:")
    for i in range(8):
        print(f"Flight {i+1}, Q-Class: {q_vars[i].X}, Y-Class: {y_vars[i].X}")
    print(f"Optimal Revenue for Spirit Airways: ${model.ObjVal}")


Optimal Allocation Decisions for Each Flight and Class:
Flight 1, Q-Class: 25.0, Y-Class: 20.0
Flight 2, Q-Class: 55.0, Y-Class: 40.0
Flight 3, Q-Class: 65.0, Y-Class: 25.0
Flight 4, Q-Class: 24.0, Y-Class: 16.0
Flight 5, Q-Class: 65.0, Y-Class: 50.0
Flight 6, Q-Class: 40.0, Y-Class: 35.0
Flight 7, Q-Class: 21.0, Y-Class: 20.0
Flight 8, Q-Class: 25.0, Y-Class: 14.0
Optimal Revenue for Spirit Airways: $203540.0


#### For Chicago-San Francisco (Flight 7):
- Q-Class Fare Price: $200
- Y-Class Fare Price: $230
- Total Additional Seats: 50

Assuming the demand is high enough to fill these seats with the same proportion of Q-Class and Y-Class passengers as in the optimal allocation, we can estimate the additional revenue for this flight leg. However, since we do not have the specific demand for this leg alone, we will assume that the extra seats could be filled entirely by Q-Class or Y-Class or a mix of both. The best-case scenario would be filling all the additional seats with the higher fare class (Y-Class).

#### Potential Additional Annual Revenue:
- Q-Class: 50 seats * $200 * 365 days = $3,650,000
- Y-Class: 50 seats * $230 * 365 days = $4,197,500

### Analysis:
- Cost of leasing the jet: $2.5 million per year.
- Additional revenue if all Q-Class: $3.65 million per year.
- Additional revenue if all Y-Class: $4.1975 million per year.

#### Request 1:
- Revenue from Elon Musk's Y-Class ticket: $420
- Revenue from his assistant's Q-Class ticket: $320
- Total Revenue for Request 1: $420 + $320 = $740

#### Request 2:
- Revenue from Layla Abbot's Q-Class ticket: $320


Comparing the two requests purely on the basis of immediate revenue, Request 1 generates more revenue ($740) compared to Request 2 ($320). Therefore, if Spirit Airways must choose only one request due to seat availability constraints, they should accept Request 1.

Additionally, airlines typically value Y-Class tickets more due to their higher profitability and the potential for additional ancillary revenue. Y-Class passengers are often business travelers who may have a higher lifetime value to the airline, as they may travel more frequently and utilize more premium services. Therefore, the long-term value of fulfilling Request 1 could also be higher for Spirit Airways.

However, if the decision were to be based on maximizing utilization of Q-Class seats specifically and if the Y-Class seats are typically in higher demand and likely to be sold to another customer later, then choosing Request 2 could make sense. But without more detailed information on demand forecasts and customer lifetime value, the decision should be based on maximizing immediate revenue, which suggests accepting Request 1. 